# Why Apache Spark?

Identify the problems Apache Spark&trade; and Databricks&reg; are well suited to solve.

## In this lesson you
* Identify the types of tasks well suited to Apache Spark’s Unified Analytics Engine.
* Identify examples of tasks not well suited for Apache Spark.

## Audience
* Primary Audience: Data Analysts
* Additional Audiences: Data Engineers and Data Scientists

## Prerequisites
* Web browser: Chrome or Firefox
* Lesson: [Getting Started]($./01-Getting-Started)
* Concept: <a href="https://www.w3schools.com/sql" target="_blank">Basic SQL</a>

-sandbox
### Getting Started

Run the following cell to configure our "classroom."

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The command `%run` runs another notebook (in this case `Classroom-Setup`), which prepares the data for this lesson.

In [3]:
%run "../Includes/Classroom-Setup"

Run the following cell to mimic a streaming data source

In [5]:
%run "../Includes/Stream-Generator"

## Lesson

### Use cases for Apache Spark
* Read and process huge files and data sets
* Query, explore, and visualize data sets
* Join disparate data sets found in data lakes
* Train and evaluate machine learning models
* Process live streams of data
* Perform analysis on large graph data sets and social networks

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Focus on learning the types of problems solved by Spark; the code examples are explained either later in this course or future courses.

-sandbox
<div>
  <div>**Apache Spark is used to...**</div>
  <div><h3 style="margin-top:0; margin-bottom:0.75em">Read and process huge files and data sets</h3></div>
</div>
Spark provides a query engine capable of processing data in very, very large data files.  Some of the largest Spark jobs in the world run on Petabytes of data.

The files in `dbfs:/mnt/training/asa/flights/all-by-year/` are stored in Azure Storage and made easily accessible using the Databricks Filesystem (dbfs).

The `%fs ls` command lists the contents of the bucket.  There are 22 comma-separated-values (CSV) files containing flight data for 1987-2008.  Spark can readily handle petabytes of data given a sufficiently large cluster.

In [9]:
%fs ls dbfs:/mnt/training/asa/flights/all-by-year/

The `CREATE TABLE` statement below registers the CSV file as a SQL Table.  The CSV file can then be queried directly using SQL.

In order to allow this example to run quickly on a small cluster, we'll use the file `small.csv` instead.

In [11]:
%sql

CREATE DATABASE IF NOT EXISTS Databricks;
USE Databricks;

CREATE TABLE IF NOT EXISTS AirlineFlight
USING CSV
OPTIONS (
  header="true",
  delimiter=",",
  inferSchema="true",
  path="dbfs:/mnt/training/asa/flights/small.csv"
);

CACHE TABLE AirlineFlight;

SELECT * FROM AirlineFlight;

-sandbox
<div>
  <div>**Apache Spark is used to...**</div>
  <div><h3 style="margin-top:0; margin-bottom:0.75em">Query, explore, and visualize data sets</h3></div>
</div>

Spark can perform complex queries to extract insights from large files and visualize the results.

The example below creates a table from a CSV file listing flight delays by airplane, counts the number of delays per model of airplane, and then graphs it.

In [14]:
%sql

CREATE TABLE IF NOT EXISTS AirlinePlane
USING csv
OPTIONS (
  header = "true",
  delimiter = ",",
  inferSchema = "false",
  path = "dbfs:/mnt/training/asa/planes/plane-data.csv"
);

CACHE TABLE AirlinePlane;

SELECT Model, count(*) AS Delays FROM AirlinePlane WHERE Model IS NOT NULL GROUP BY Model ORDER BY Delays DESC LIMIT 10;

-sandbox
<div>
  <div>**Apache Spark is used to...**</div>
  <div><h3 style="margin-top:0; margin-bottom:0.75em">Join disparate data sets found in data lakes</h3></div>
</div>

Companies frequently have thousands of large data files gathered from various teams and departments, typically using a diverse variety of formats, including CSV, JSON and XML.

These are called Data Lakes. Data Lakes differ from Data Warehouses in that they don't require someone to spend weeks or months preparing a unified enterprise schema and then populating it.

Frequently an analyst wishes to run simple queries across various data files, without taking the time required to construct a fully-fledged Data Warehouse.

Spark excels in this type of workload by enabling users to simultaneously query files from many different storage locations, and then formats and join them together using Spark SQL.

Spark can later load the data into a Data Warehouse if desired.

In [16]:
%sql

SELECT p.manufacturer AS Manufacturer,
       avg(depDelay) AS Delay
FROM AirlinePlane p
JOIN AirlineFlight f ON p.tailnum = f.tailnum
WHERE p.manufacturer IS NOT null
GROUP BY p.manufacturer
ORDER BY Delay DESC
LIMIT 10

Not only does Spark bring together files from many different locations, it also brings in disparate data sources and file types such as:
* JDBC Data Sources like SQL Server, Azure SQL Database, MySQL, PostgreSQL, Oracle,  etc.
* Parquet files
* CSV files
* ORC files
* JSON files
* HDFS file systems
* Apache Kafka
* And with a little extra work, Web Services Endpoints, TCP-IP sockets, and just about anything else you can imagine!

-sandbox
<div>
  <div>**Apache Spark is used to...**</div>
  <div><h3 style="margin-top:0; margin-bottom:0.75em">Train and evaluate machine learning models</h3></div>
</div>

Spark performs predictive analytics using machine learning algorithms.

The example below trains a linear regression model using past flight data to predict delays based on the hour of the day.

In [19]:
from pyspark.sql.functions import col, floor, translate, round
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, OneHotEncoder
from pyspark.ml.regression import LinearRegression

inputDF = (spark.read.table("AirlineFlight")
  .withColumn("HourOfDay", floor(col("CRSDepTime")/100))
  .withColumn("DepDelay", translate(col("DepDelay"), "NA", "0").cast("integer")))

(trainingDF, testDF) = inputDF.randomSplit([0.80, 0.20], seed=999)

pipeline = Pipeline(stages=[
    OneHotEncoder(inputCol="HourOfDay", outputCol="HourVector"),
    VectorAssembler(inputCols=["HourVector"], outputCol="Features"),
    LinearRegression(featuresCol="Features", labelCol="DepDelay", predictionCol="DepDelayPredicted", regParam=0.0)
  ])

model = pipeline.fit(trainingDF)
resultDF = model.transform(testDF)

displayDF = resultDF.select("Year", "Month", "DayOfMonth", "CRSDepTime", "UniqueCarrier", "FlightNum", "DepDelay", round("DepDelayPredicted", 2).alias("DepDelayPredicted"))
display(displayDF)

In [20]:
display(
  resultDF
    .groupBy("HourOfDay")
    .avg("DepDelay", "DepDelayPredicted")
    .toDF("HourOfDay", "Actual", "Predicted")
    .orderBy("HourOfDay")
)

-sandbox
<div>
  <div>**Apache Spark is used to...**</div>
  <div><h3 style="margin-top:0; margin-bottom:0.75em">Process live streams of data</h3></div>
</div>

Besides aggregating static data sets, Spark can also process live streams of data such as:
* File Streams
* TCP-IP Streams
* Apache Kafka
* Custom Streams like Twitter & Facebook

Before processing streaming data, a data source is required.

The cell below first deletes any temp files, and then generates a stream of fake flight data for up to 30 minutes.

In [23]:
%scala

// Clean any temp files from previous runs.
DummyDataGenerator.clean()

// Generate data for 5 minutes.
// To force it to stop rerun with 0.
DummyDataGenerator.start(5)

The example below connects to and processes a fake, fire-hose of flight data by:
0. Reading in a stream of constantly updating CSV files.
0. Parsing the flight's date and time.
0. Computing the average delay for each airline based on the most recent 15 seconds of flight data.
0. Plotting the results in near-real-time.

**Disclaimer:** The real-time data represented here is completely fictional and is not intended to reflect actual airline performance.

In [25]:
from pyspark.sql.functions import col, date_format, unix_timestamp, window
from pyspark.sql.types import StructType

spark.conf.set("spark.sql.shuffle.partitions", "8")

flightSchema = (StructType()
  .add("FlightNumber", "integer")
  .add("DepartureTime", "string")
  .add("Delay", "double")
  .add("Airline", "string")
)
streamingDF = (spark.readStream
  .schema(flightSchema)
  .csv(DummyDataGenerator.streamDirectory)
  .withColumn("DepartureTime", unix_timestamp("DepartureTime", "yyyy-MM-dd'T'HH:mm:ss").cast("timestamp"))
  .withWatermark("DepartureTime", "5 minute")
  .groupBy( window("DepartureTime", "15 seconds"), "Airline" )
  .avg("Delay")
  .select(col("window.start").alias("Start"), "Airline", col("avg(delay)").alias("Average Delay"))
  .orderBy("start", "Airline")
  .select(date_format("start", "HH:mm:ss").alias("Time"), "Airline", "Average Delay")
)
display(streamingDF)

-sandbox
<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Remember to stop your stream by clicking the **Cancel** link up above.

-sandbox
<div>
  <div>**Apache Spark is used to...**</div>
  <div><h3 style="margin-top:0; margin-bottom:0.75em">Perform analysis on large graph data sets and social networks</h3></div>
</div>

The open source <a href="https://graphframes.github.io/" target="_blank">GraphFrames</a> library extends Spark to study not the data itself, but the network of relationships between entities.  This facilitates queries such as:
* **Shortest Path:** What is the shortest route from Springfield, IL to Austin, TX?
* **Page Rank:** Which airports are the most important hubs in the USA?
* **Connected Components:** Find strongly connected groups of friends on Facebook.
* (just to name a few)

#### Connected Graphs

The example below is a visualization of a network of airports connected by flight routes.

Databricks can display this network using popular third-party visualization library such as:
* <a href="https://d3js.org/" target="_blank">D3.js - Data-Driven Documents</a>
* <a href="https://matplotlib.org/" target="_blank">Matplotlib: Python plotting</a>
* <a href="http://ggplot.yhathq.com/" target="_blank">ggplot</a>
* <a href="https://plot.ly/" target="_blank">Plotly<a/>

-sandbox
<iframe style='border-style:none; position:absolute; left:-150px; width:1170px; height:700px'
          src='https://mbostock.github.io/d3/talk/20111116/#14'
/>

-sandbox
#### PageRank algorithm
The <a href="https://en.wikipedia.org/wiki/PageRank" target="_blank">PageRank</a> algorithm, named after Google co-founder Larry Page, assesses the importance of a hub in a network.

The example below uses the <a href="https://graphframes.github.io/" target="_blank">GraphFrames</a> API  to compute the **PageRank** of each airport in the United States and shows the top 10 most important US airports.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> This example requires the GraphFrames library that may not have been setup on your cluster.  Read the example below rather than running it.

In [31]:
from pyspark.sql.functions import col, concat_ws, round
from graphframes import GraphFrame

flightVerticesDF = (spark.read
  .option("header", True)
  .option("delimiter", "\t")
  .csv("dbfs:/mnt/training/asa/airport-codes/airport-codes.txt")
  .withColumnRenamed("IATA", "id"))

flightEdgesDF = (spark.table("Databricks.AirlineFlight")
  .withColumnRenamed("Origin", "src")
  .withColumnRenamed("Dest", "dst"))

flightGF = GraphFrame(flightVerticesDF, flightEdgesDF)
pageRankDF = flightGF.pageRank(tol=0.05)

resultsDF = (pageRankDF.vertices
  .select(concat_ws(", ", col("city"), col("state")).alias("Location"),
          round(col("pagerank"), 1).alias("Rank"))
  .orderBy(col("pagerank").desc()))

display(resultsDF)

## Review
**Question:** Which of the following are good applications for Apache Spark? (Select all that apply.)
0. Querying, exploring, and analyzing very large files and data sets
0. Joining data lakes
0. Machine learning and predictive analytics
0. Processing streaming data
0. Graph analytics
0. Overnight batch processing of very large files
0. Updating individual records in a database

**Answer:** All but #7. Apache Spark uses SQL to read and performs analysis on large files, but it is not a Database.

## Additional Topics & Resources

**Q:** What makes Spark different than Hadoop?
**A:** Spark on Databricks performs 10-2000x faster than Hadoop Map-Reduce.  It does this by providing a high-level query API which allows Spark to highly optimize the internal execution without adding complexity for the user.  Internally, Spark employs a large number of optimizations such as pipelining related tasks together into a single operation, communicating in memory, using just-in-time code generation, query optimization, efficient tabular memory (Tungsten), caching, and more.

**Q:** What are the visualization options in Databricks?
**A:** Databricks provides a wide variety of <a href="https://docs.databricks.com/user-guide/visualizations/index.html" target="_blank">built-in visualizations</a>.  Databricks also supports a variety of 3rd party visualization libraries, including <a href="https://d3js.org/" target="_blank">d3.js</a>, <a href="https://matplotlib.org/" target="_blank">matplotlib</a>, <a href="http://ggplot.yhathq.com/" target="_blank">ggplot</a>, and <a href="https://plot.ly/" target="_blank">plotly<a/>.

**Q:** Where can I learn more about DBFS?
**A:** See the document <a href="https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html" target="_blank">Databricks File System - DBFS</a>.

**Q:** Where can I find a list of the machine learning algorithms supported by Spark?
**A:** The Spark documentation for Machine Learning describes the algorithms for classification, regression, clustering, recommendations (ALS), neural networks, and more.  The documentation doesn't provide a single consolidated list, but by browsing through the <a href="http://spark.apache.org/docs/latest/ml-guide.html" target="_blank">Spark MLLib documentation</a> you can find the supported algorithms.  Additionally, <a href="https://spark-packages.org/" target="_blank">3rd party libraries</a> provide even more algorithms and capabilities.

**Q:** Where can I learn more about stream processing in Spark?
**A:** See the <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html" target="_blank">Structured Streaming Programming Guide</a>.

**Q:** Where can I learn more about GraphFrames?
**A:** See the <a href="http://graphframes.github.io/" target="_blank">GraphFrames Overview</a>.  The Databricks blog has an <a href="https://databricks.com/blog/2016/03/16/on-time-flight-performance-with-graphframes-for-apache-spark.html">example</a> which uses d3 to perform visualizations of GraphFrame data.